In [32]:
import unittest
from unittest.mock import patch, MagicMock, Mock
import requests

class SwitchBox():
    def __init__(self, url) -> None:
        super().__init__()
        self.url = url
        self.info = {}
        self.relays = []
        # self.load()
    def temp_load(self):
        response = requests.get(self.url)
        self._on = response.json().get('on', -1)

    def load(self):
        response = requests.get(self.url + "/info")
        self.info = response.json() # .get('device', {})

        # response = requests.get(self.url + "/state")
        # self.state = response.json().get('relays', [])

    # @property
    # def on(self):
    #     if len(self.relays):
    #         self._on = self.relays[0]['state']
    #     return self._on

    # @on.setter
    # def on(self, value):
    #     self._on = value
    #     if len(self.relays):
    #         self.relays[0]['state'] = self._on
    #     requests.post(self.url + f"/s/{value}")

    @property
    def on(self):
        response = requests.get(self.url)
        self._on = response.json().get('on', 0)
        return self._on

    @on.setter
    def on(self, value):
        requests.post(self.control_url, json={'on': value})
        self._on = value    

switch_box_info = {
    "device": {
        "deviceName": "My switchBox LIGHT",
        "type": "switchBox",
        "product": "switchBoxLight",
        "hv": "s_swBL.1.0",
        "fv": "0.1021",
        "universe": 0,
        "apiLevel": "20200831",
        "iconSet": 38,
        "categories": [
            3,
            2
        ],
        "id": "8ece4ee6d15c",
        "ip": "192.168.0.197",
        # "availableFv": None
    }
}

switch_box_state = {
    "relays": [
        {
            "relay": 0,
            "state": 0
        }
    ]
}

def my_function(url):
    response = requests.get(url)
    return response.json()

class TestSwitchBox(unittest.TestCase):

    @patch('requests.get')
    def test_switchbox_init(self, mock_get: Mock):

        # Mock the SmartSwitch state response
        # mock_switch_box_info = MagicMock()
        # mock_switch_box_info.json.return_value = {'device': switch_box_info['device']}

        # mock_switch_box_state = MagicMock()
        # mock_switch_box_state.json.return_value = switch_box_state

        # Simulate successful communication initially
#        mock_get.return_value = MagicMock(status_code=200, json=lambda: switch_box_info)

        # mock_get.return_value = "hi"
        mock_get.return_value.json = lambda: {"on":"hahhaha"}
        def my_side_effect(url, *args, **kwargs):
            if "fd" in url:
                mock = Mock()
                # mock.json = lambda: {"on":"hihiihi"}
                # mock.json = lambda: {'relays': [{'state': 0}]}
                mock.json = lambda: {"on":0}
                return mock
            else:
                mock = Mock()
                #mock.json = lambda: {'relays': [{'state': 1}]}
                mock.json = lambda: {"on":15}
                return mock
        mock_get.side_effect = my_side_effect
        my_switch = SwitchBox("129932190392")
        my_switch.temp_load()
        print(my_switch._on)
        # xd = my_function("fdsfdsfd")
        # print (xd)

        # xd = my_function("ziziziz")
        # print (xd)        
        # mock_get.return_value = MagicMock(status_code=200, json=lambda: {"deviceName": "My switchBox LIGHT"})

        # mock_get.return_value = mock_switch_box_info
        # def side_effect(*args, **kwargs):
        #     return -3
        
        # def mock_get_side_effect(url, *args, **kwargs):
        #     if "/info" in url:
        #         return mock_switch_box_info
        #     else:
        #         return mock_switch_box_state

        # mock_get.side_effect = side_effect

        switch_box = SwitchBox("http://192.168.0.197")
        # switch_box.load()
        # switch_box.temp_load()

        print(switch_box.on)
        self.assertEqual(switch_box.on, 15)
        #print(switch_box.info)

        self.assertTrue(switch_box.info["deviceName"] == "My switchBox LIGHT")
        self.assertIsInstance(switch_box.info["deviceName"], str)
        self.assertIsInstance(switch_box.info["type"], str)
        self.assertIsInstance(switch_box.info["hv"], str)
        self.assertIsInstance(switch_box.info["fv"], str)
        self.assertIsInstance(switch_box.info["universe"], int)
        self.assertIsInstance(switch_box.info["apiLevel"], str)

#        self.assertIsInstance(switch_box.state[0], dict)
#        self.assertEqual(switch_box.on, 0)        

def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestSwitchBox('test_switchbox_init'))
    # suite.addTest(TestSwitchBox('test_switchbox_call'))
    # suite.addTest(TestSwitchBox('test_switchbox_no_device'))
    # suite.addTest(TestSwitchBox('test_switch_broken_communication'))
    return suite


if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())



15


KeyboardInterrupt: 